<a href="https://colab.research.google.com/github/YasmineSarraj/MLOPS-assignments/blob/main/Assignment%203/MLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
! pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
! pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.6-py3-none-any.whl size=143237 sha256=f5fcb41c945caddaa2b30a93670782523efb632e5d6a2a4efd8d338ee7418ad4
  Stored in directory: /root/.cache/pip/wheels/0

In [3]:
import mlflow

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score, mean_squared_error
from statistics import mean

In [7]:
penguins = pd.read_csv("https://github.com/allisonhorst/palmerpenguins/raw/5b5891f01b52ae26ad8cb9755ec93672f49328a8/data/penguins_size.csv")
penguins.dropna(inplace=True)

attributes = penguins.iloc[:,2:6]
scaler = StandardScaler()
num = scaler.fit_transform(attributes)
data_n = pd.DataFrame(num, columns=['culmen_length_mm','culmen_depth_mm','flipper_length_mm','body_mass_g'])

species = penguins.iloc[:, 0]
labelencoder = LabelEncoder()
cat = labelencoder.fit_transform(species)
enc_species = pd.DataFrame(cat, columns=['species_short'])

enc_species = pd.get_dummies(species)


#Tensor Of all the numerical values, lenghth etc. 
X_tensor = torch.tensor(data_n.astype(np.float32).values)

#Tensor of the encoded categorical variable
y_tensor = torch.tensor(enc_species.astype(np.float32).values)



In [4]:
mlflow.set_experiment("Penguins ANN")
experiment_id = mlflow.get_experiment_by_name('Penguins ANN').experiment_id

2023/04/19 15:20:31 INFO mlflow.tracking.fluent: Experiment with name 'Penguins ANN' does not exist. Creating a new experiment.


In [13]:
with mlflow.start_run(run_name="run with artifact", nested=True, experiment_id=experiment_id):
  # Your machine learning code here
  # Initializing Hyperparameters
  epochs = 50
  learning_rate = 0.001
  loss_set = {}
  # 1. Creating a FeedForwardNetwork
  model_net4 = torch.nn.Sequential(
                            # Input layer
                            torch.nn.Linear(4,8),
                            torch.nn.ReLU(),
                            # 1. Hidden layer
                            torch.nn.Linear(8,16),
                            torch.nn.ReLU(),
                            # 2. Hidden layer
                            torch.nn.Linear(16,32),
                            torch.nn.ReLU(),
                            # 3. Hidden layer
                            torch.nn.Linear(32,64),
                            torch.nn.ReLU(),
                            # Output layer
                            torch.nn.Linear(64,3),
                            torch.nn.Softmax()  # We have a multiclass single-label classification problem, since a penguin can only be from one species
                          );
  loss_cel = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.RMSprop(model_net4.parameters(), lr=learning_rate)

  # Loop over the number of epochs
  for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
    lost_set_epoch = {}
    j = 0
    for x, y_t in zip(X_tensor, y_tensor):
        output = model_net4.forward(x.float())
        # 2. FeedForward Evaluation
        loss = loss_cel(output, y_t.float())
        optimizer.zero_grad();

        # 3. Backward / Gradient Calculation
        loss.backward()
        # 4. Back Propagation
        optimizer.step()
        # Store the loss for each sample of data
        lost_set_epoch[j] = loss
        j = j + 1
    
    # Store the loss for each epoch
    loss_set[i] = torch.mean(torch.stack(list(lost_set_epoch.values()))).detach().numpy()

    # Display the loss after every 10 epochs
    if (i % 10)==0:
      print (f"Loss: {loss_set[i]}")

  mlflow.log_param("Learning rate", learning_rate)
  mlflow.log_param("Num. of epochs", epochs)

  from pandas.core.groupby.groupby import OutputFrameOrSeries
  with torch.no_grad():
    n_correct = 0
    n_samples = len(y_tensor)

    outputs = model_net4.forward(X_tensor)
    for idx, output in enumerate(outputs):
      pred = np.argmax(output).item()
      true_val = np.argmax(y_tensor[idx]).item()
      
      if pred == true_val:
        n_correct += 1

    acc = 100 * (n_correct / n_samples)

  mlflow.log_metric("accuracy", acc)
  params_path = "/content/model.pt"
  torch.save(model_net4.state_dict(), params_path)
  mlflow.log_artifact(params_path)
  
  # End the current run
  mlflow.end_run()

<ipython-input-13-3390ebd6c26f>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):


  0%|          | 0/50 [00:00<?, ?epoch/s]

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Loss: 0.7579784393310547
Loss: 0.557512104511261
Loss: 0.5582020282745361
Loss: 0.5606886148452759
Loss: 0.5578663945198059


In [12]:
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")
## Step 3 - Installing pyngrok for remote tunnel access using ngrock.com
!pip install pyngrok --quiet
from pyngrok import ngrok
from getpass import getpass
# Terminate open tunnels if any exist
ngrok.kill()
## Step 4 - Login on ngrok.com and get your authtoken from https://dashboard.ngrok.com/auth
# Enter your auth token when the code is running
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

Enter the ngrok authtoken: ··········


MLflow Tracking UI: https://b281-35-192-48-155.ngrok-free.app
